In [ ]:
1. df['Rating_4-5'] = df['Item_Rating'].between(4,5,inclusive=True).astype(int) (For playing with ratings)(Binning them)

2. # used to handle the skewness in the data
   df['SalesInMillions'] = np.log1p(df['SalesInMillions']) (Handling skewness with log transformation)
   sns.distplot(df['SalesInMillions']) (Plotting to see skewness)
   y_pred1=np.expm1(y_pred1) (inverse tranforming the output results)
    
3. #code to draw a heatmap
   corr = df.corr()
   sns.heatmap(corr,annot=True)

3. # In order to split a string a string in the columns and to get specific characters out from it
   banned = ['out','of','5','stars']
   f = lambda x:' '.join([item for item in x.split() if item not in banned])
   df['Reviews'] = df['Reviews'].apply(f)
   df['Reviews']

4. If there are 5-10 or lesser big outliers in the data then remove them without any second thoughts as they will highly sway  
   your results.

5. # using sweeetviz to do initial data analysis
   my_report = sweetviz.compare([df.drop(['YEAR','ID'],axis=1), "Train"], [test.drop(['YEAR','ID'],axis=1), "Test"],              "SalesInMillions")
   my_report.show_html('Report.html')

6. #website for regular expressions tutorial
   https://www.coursera.org/learn/python-text-mining/lecture/sVe8B/regular-expressions
   new_df['just_movie_titles'] =        df['movie_title'].str.extract('(.+?) \(')


7. #finding index of all the rows with some null values
   inds = pd.isnull(df).any(1).nonzero()[0]

8. #way to find all the Null Values
   df.info()
   df.isnull().sum(axis=0)

9. #finding error keys
   import sklearn
   sorted(sklearn.metrics.SCORERS.keys()) 

10. #reading the dataframe back to csv file
    pred_final2 = pd.DataFrame(gsearch5.best_estimator_.predict_proba(df_final))
    pred_final2.to_excel('ODI_Participants_data/Sample_submission.xlsx')
    pred_final2.head()

11. #using SMOTE from imblearn for imbalanced classes
    X = df.drop('Cover_Type',axis=1)
    Y = df['Cover_Type']
    X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)
    sm = SMOTE(random_state=0)
    X_train,y_train = sm.fit_sample(X_train,y_train)

12. #seed samppling for xgboost
    preds = 0
    for seed_val in range(100):
        print (seed_val)
        m= XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=17,
                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=4, scale_pos_weight=1, seed=seed_val)
        m.fit(X,Y)
        predict=m.predict(test)
        preds += predict
    preds = pd.DataFrame(preds/100)

13. #preprocessing outliers using wincerization
    upperbound , lowerbound = np.percentile(x,[1,99])
    y = np.clip(x,upperbound,lowerbound)
    pd.Series(y).hist(bins=30) #simplt shows the new distribution

14. #rank transformation can be used in place of min max scaling as min max scaling is affected by outliers
15. #use frequency encoding for categorical data as it'll tell how many times a feature occured. Don't use it when features have same frequency
16. #use x.value_counts(),x.isnull(),x.dtypes for looking into the dataset
17. #label encoding and one hot encoding
    o_e = OneHotEncoder(sparse=False)
    l_e = LabelEncoder()
    df['Team_Venue'] = l_e.fit_transform(df['Team1_Venue'])
    oh_cols_train = pd.DataFrame(o_e.fit_transform(df[['Team1','Team_Venue','Team1_Innings','Team2']])) 
    oh_cols_train.drop(columns=[0,16],inplace=True)
    df = pd.concat([oh_cols_train,df],axis=1)
18. #playing with dates
    rr = pd.read_csv('RR.csv')
    rr['hour_min'] = rr['time'].str.extract('(\d{2}:\d{2})')
    rr['time'] = pd.to_datetime(rr['time'], format='%H:%M:%S').dt.time
    rr['hour'] = pd.to_datetime(rr['hour_min'], format='%H:%M').dt.hour
    rr['min'] = pd.to_datetime(rr['hour_min'], format='%H:%M').dt.minute
    #extracting year from date
    df_m['listing_year'] = pd.to_datetime(df_m['listing_date']).dt.year
    df_m['issue_year'] = pd.to_datetime(df_m['issue_date']).dt.year
19. #selecting categorical and numeric columns from a dataset
    cat_colc = list(train.select_dtypes(include=['object']).columns)
    num_colc = list(train.select_dtypes(exclude=['object']).columns)

In [1]:
#parameter tuning for xgboost
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='neg_root_mean_squared_error',n_jobs=-1, cv=5)
gsearch1.fit(X,Y)
gsearch1.best_params_,gsearch1.best_score_

param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=gsearch1.best_params_['max_depth'],
 min_child_weight=gsearch1.best_params_['min_child_weight'], gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test2, scoring='f1_weighted',n_jobs=-1, cv=5)
gsearch2.fit(X,Y1)
gsearch2.best_params_,gsearch2.best_score_

param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=gsearch1.best_params_['max_depth'],
 min_child_weight=gsearch1.best_params_['min_child_weight'], gamma=gsearch2.best_params_['gamma'], subsample=0.8, colsample_bytree=0.8, nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='f1_weighted',n_jobs=-1, cv=5)
gsearch3.fit(X,Y1)
gsearch3.best_params_,gsearch3.best_score_

param_test4 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],'learning_rate':[i/10.0 for i in range(0,5)]}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=gsearch1.best_params_['max_depth'],
 min_child_weight=gsearch1.best_params_['min_child_weight'], gamma=gsearch2.best_params_['gamma'], subsample=gsearch3.best_params_['subsample'], colsample_bytree=gsearch3.best_params_['colsample_bytree'], nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='f1_weighted',n_jobs=-1, cv=5)
gsearch4.fit(X,Y1)
gsearch4.best_params_,gsearch4.best_score_

SyntaxError: invalid syntax (<ipython-input-1-3cf04a790655>, line 33)

In [ ]:
# various libraries in python I have used till now
# preprocessing and paramtere tuning libraries
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV 
from sklearn.metrics import mean_squared_log_error

# model
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from skleanrn.linear_model import LinearRegression

import sweetviz

In [1]:
#playing with files in python

with open(filename) as training_file:
        reader = csv.reader(training_file, delimiter=',')    
        imgs = []
        labels = []

        next(reader, None)
        
        for row in reader:
            label = row[0]
            data = row[1:]
            img = np.array(data).reshape((28, 28))

            imgs.append(img)
            labels.append(label)

        images = np.array(imgs).astype(float)
        labels = np.array(labels).astype(float)
            

NameError: name 'filename' is not defined

In [ ]:
#seed sampling
preds = 0
for seed_val in range(100):
    print (seed_val)
    m= XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=5, gamma=0.0, subsample=0.8, colsample_bytree=0.8, nthread=4, scale_pos_weight=1, seed=seed_val)
    m.fit(X,Y)
    predict=m.predict(test)
    preds+=np.expm1(predict)
preds = pd.DataFrame(preds/100)

#importtant courses or places for revision
1. (#decision tree explanation)https://courses.analyticsvidhya.com/courses/take/getting-started-with-decision-trees/lessons/8157460-terminologies-related-to-decision-trees
2. how to win kaggle competitions(coursera)
3. data science specialization from michigan university(coursera)
4. Deep learning specialization by deeplearning.ai(coursera)
5. applied tensorflow specialization by deeplearning.ai(coursera)
6. https://www.analyticsvidhya.com/blog/2016/01/guide-data-exploration/
7. https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/
8. https://sebastianraschka.com/Articles/2014_about_feature_scaling.html
9. https://github.com/kunalj101/Data-Science-Hacks?utm_source=blog&utm_medium=git-github-essential-guide-beginners
10. https://stats.stackexchange.com/questions/107610/what-is-the-reason-the-log-transformation-is-used-with-right-skewed-distribution#:~:text=Correspondingly%2C%20if%20you%20apply%20the%20log-transformation%20to%20something,So%20the%20log%20transformation%20wouldn%27t%20be%20helpful%20then.
11. https://www.kaggle.com/sudalairajkumar/simple-feature-engg-notebook-spooky-author

In [ ]:
def baseliner(X, y, cv=3, metric='neg_log_loss'):
    print("Baseliner Models\n")
    eval_dict = {}
    models = [lgb.LGBMClassifier(), xgb.XGBClassifier(), GradientBoostingClassifier(),
                  LogisticRegression(), GaussianNB(), RandomForestClassifier(), DecisionTreeClassifier(),
                  ExtraTreeClassifier(), AdaBoostClassifier(), BaggingClassifier(), ExtraTreesClassifier(),
              SVC(probability=True), KNeighborsClassifier() 
                 ]
    print("Model Name \t |   CV")
    print("--" * 50)

    for index, model in enumerate(models, 0):
        model_name = str(model).split("(")[0]
        eval_dict[model_name] = {}

        results = cross_val_score(model, X, y, cv=cv, scoring=metric)
        eval_dict[model_name]['cv'] = results.mean()

        print("%s \t | %.4f \t" % (
            model_name[:12], eval_dict[model_name]['cv']))